In [1]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

In [2]:
def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

In [3]:
def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size

In [4]:
def mask(image):
    r = (image==255)
    return r

In [5]:
def ELBP_CI(image,beta):
    return (image>beta)

In [6]:
def get_pixel_else_0(l, idx, idy, default=0):
    try:
        return l[idx,idy]
    except IndexError:
        return default

def find_variations(pixel_values):
    prev = pixel_values[-1]
    t = 0
    for p in range(0, len(pixel_values)):
        cur = pixel_values[p]
        if cur != prev:
            t += 1
        prev = cur
    return t    

def CLBP_M(img,c,encoding='uniform'):
    clbpm = np.zeros_like(img,dtype=np.float64)
    
    for x in range(0, len(clbpm)):
        for y in range(0, len(clbpm[0])):
            center = img[x,y]
            top_left      = abs(int(get_pixel_else_0(img, x-1, y-1))-int(center))
            top_up        = abs(int(get_pixel_else_0(img, x, y-1)) - int(center))
            top_right     = abs(int(get_pixel_else_0(img, x+1, y-1))-int(center))
            right         = abs(int(get_pixel_else_0(img, x+1, y )) -int(center))
            left          = abs(int(get_pixel_else_0(img, x-1, y )) -int(center))
            bottom_left   = abs(int(get_pixel_else_0(img, x-1, y+1))-int(center))
            bottom_right  = abs(int(get_pixel_else_0(img, x+1, y+1))-int(center))
            bottom_down   = abs(int(get_pixel_else_0(img, x,   y+1 ))-int(center))
            
            t = lambda x,c: (x>=c)*1
            
            v1 = t(right,c)
            v2 = t(top_right,c)
            v3 = t(top_up,c)
            v4 = t(top_left,c)
            v5 = t(left,c)
            v6 = t(bottom_left,c)
            v7 = t(bottom_down,c)
            v8 = t(bottom_right,c)
            
            va = [v1,v2,v3,v4,v5,v6,v7,v8]
            
            if(encoding=='uniform'):
                if(find_variations(va)<=2):
                    clbpm.itemset((x,y), va.count(0))
                else :
                    clbpm.itemset((x,y),9)
            else:
                weights = [1, 2, 4, 8,16,32,64,128]
                res = 0
                for a in range(0, len(weights)):
                    res += weights[a] * va[a]
                clbpm.itemset((x,y),res)
    
    return clbpm

def calc_c(img,img1,size):
    ctemp  =  0
    beta = 0
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            if(img1[x,y]!=255):
                c = 0
                center = img[x,y]
                beta+=center
                c+= abs(int(get_pixel_else_0(img, x-1, y-1))-int(center))
                c+=abs(int(get_pixel_else_0(img, x, y-1)) - int(center))
                c+= abs(int(get_pixel_else_0(img, x+1, y-1))-int(center))
                c+= abs(int(get_pixel_else_0(img, x+1, y )) -int(center))
                c+= abs(int(get_pixel_else_0(img, x-1, y )) -int(center))
                c+= abs(int(get_pixel_else_0(img, x-1, y+1))-int(center))
                c+= abs(int(get_pixel_else_0(img, x+1, y+1))-int(center))
                c+= abs(int(get_pixel_else_0(img, x,   y+1 ))-int(center))
                c/=8
                ctemp+=c
    ctemp/=size
    beta/=size
    return ctemp,beta


def Completed_LBP(images,dim,sizes,alpha=1,encoding='uniform'):
    hist_s = {}
    hist_m = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    print(k,i,j)
                
                    s = sizes['c'+str(k)+'_'+i+'_'+j]
                    c,beta = calc_c(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],images['c'+str(k)+'_'+i+'_'+j][:,:,0],s)
                    c*=alpha
                    
                    masc = ELBP_CI(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],beta)
                    
                    clbp_pens = CLBP_M(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],c,encoding)
                    lbp_pens = local_binary_pattern(images['c'+str(k)+'_'+i+'_'+j][:,:,dim], 8, 1, encoding)
                    
                    if(encoding=='uniform'):
                        clbp_pens+=10*masc
                        lbp_pens+=10*masc
                    else:
                        clbp_pens+=256*masc
                        lbp_pens+=256*masc
                        
                    clbp_pens+=(mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*1000)
                    lbp_pens+=(mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*1000)
                    
                    if(encoding=='uniform'):
                        hist_clbp,_ = np.histogram(clbp_pens.ravel(),range(0,21))
                        hist_lbp,_ = np.histogram(lbp_pens.ravel(),range(0,21))
                    else:
                        hist_clbp,_ = np.histogram(clbp_pens.ravel(),range(0,513))
                        hist_lbp,_ = np.histogram(lbp_pens.ravel(),range(0,513))
                        
                    hist_s['c'+str(k)+'_'+i+'_'+j] = hist_lbp/s
                    hist_m['c'+str(k)+'_'+i+'_'+j] = hist_clbp/s
                except:
                    break
                   
    return hist_s,hist_m

In [7]:
def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def compute_dist(param1,param2,n1,n2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def distance_distribution(features,sizes):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
    print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [8]:
images = load_images()
sizes = n_size(images)

# using riu2 encoding

In [9]:
hist_Ys,hist_Ym = Completed_LBP(images,0,sizes)
hist_Cbs,hist_Cbm = Completed_LBP(images,1,sizes)
hist_Crs,hist_Crm = Completed_LBP(images,2,sizes)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2

20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2
26 1 3
26 1 4
26 1 5
26 1 6
26 2 1
26 2 2
26 2 3
26 2 4
27 1 1
27 1 2
27 1 3
27 1 4
27 1 5
27 2 1
27 2 2
27 2 3
27 2 4
27 2 5
28 1 1
28 1 2
28 1 3
28 1 4
28 1 5
28 2 1
28 2 2
28 2 3
28 2 4
29 1 1
29 1 2
29 1 3
29 1 4
29 1 5
29 1 6
29 1 7
29 2 1
29 2 2
29 2 3
30 1 1
30 1 2
30 1 3
30 1 4
30 1 5
30 2 1
30 2 2
30 2 3
30 2 4
31 1 1
31 1 2
31 1 3
31 1 4
31 1 5
31 1 6
31 1 7
31 2 1
31 2 2
31 2 3
32 1 1
32 1 2
32 1 3
32 1 4
32 1 5
32 1 6
32 1 7
32 2 1
32 2 2
32 2 3
32 2 4
33 1 1
33 1 2
33 1 3
33 1 4
33 1 5
33 1 6
33 2 1
33 2 2
33 2 3
33 2 4
34 1 1
34 1 2
34 1 3
34 1 4
34 1 5
34 1 6
34 1 7
34 2 1
34 2 2
34 2 3
35 1 1
35 1 2
35 1 3
35 1 4

In [13]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Ys,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cbs,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Crs,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
33.72809302683847 28.548094074495328 22.186542463713014


In [14]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Ym,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cbm,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Crm,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
26.578002724145747 16.825032215508898 23.268720330969504


In [17]:
Y = pd.DataFrame(hist_Ys)
Cb = pd.DataFrame(hist_Cbs)
Cr = pd.DataFrame(hist_Crs)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CLBP_S|c_riu2.csv',sep = ',')



In [15]:
Y = pd.DataFrame(hist_Ym)
Cb = pd.DataFrame(hist_Cbm)
Cr = pd.DataFrame(hist_Crm)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CLBP_M|c_riu2.csv',sep = ',')

# using default encoding

In [19]:
hist_Ys,hist_Ym = Completed_LBP(images,0,sizes,encoding='default')
hist_Cbs,hist_Cbm = Completed_LBP(images,1,sizes,encoding='default')


1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2

TypeError: Completed_LBP() got an unexpected keyword argument 'rncoding'

In [20]:
hist_Crs,hist_Crm = Completed_LBP(images,2,sizes,encoding='default')

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [21]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Ys,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cbs,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Crs,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
24.556249271970085 27.55788432199898 21.77575563796077


In [23]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Ym,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cbm,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Crm,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
25.31798728864151 19.704795926411876 22.017939059625025


In [24]:
Y = pd.DataFrame(hist_Ys)
Cb = pd.DataFrame(hist_Cbs)
Cr = pd.DataFrame(hist_Crs)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CLBP_S|c_default.csv',sep = ',')

In [25]:
Y = pd.DataFrame(hist_Ym)
Cb = pd.DataFrame(hist_Cbm)
Cr = pd.DataFrame(hist_Crm)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CLBP_M|c_default.csv',sep = ',')

# for different values of alpha

In [24]:

distances_Y,distances_Cb,distances_Cr = [],[],[]

for alpha in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    start = time.time()
    
    hist_Y = Completed_LBP(images1,images2,0,sizes,alpha)
    hist_Cb = Completed_LBP(images1,images2,1,sizes,alpha)
    hist_Cr = Completed_LBP(images1,images2,2,sizes,alpha)

    same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
    same_pens_Y = [ v for v in same_pens_Y. values() ]
    diff_pens_Y = [ v for v in diff_pens_Y. values() ]

    same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
    same_pens_Cb = [ v for v in same_pens_Cb. values() ]
    diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

    same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
    same_pens_Cr = [ v for v in same_pens_Cr. values() ]
    diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

    dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
    dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
    dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)
    
    print('TIME TAKEN FOR ALPHA = {} is {}sec'.format(alpha,time.time()-start))
    print(' Y : {} , Cb : {} , Cr : {} '.format(dy,dcb,dcr))
    distances_Y.append(dy)
    distances_Cb.append(dcb)
    distances_Cr.append(dcr)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


TIME TAKEN FOR ALPHA = 0.1 is 508.89008688926697sec
 Y : 23.136190426840418 , Cb : 16.632201709853998 , Cr : 18.917657743195594 
TIME TAKEN FOR ALPHA = 0.2 is 527.9413475990295sec
 Y : 23.907366618476605 , Cb : 6.9480118687531505 , Cr : 7.188929776803303 
TIME TAKEN FOR ALPHA = 0.3 is 518.519608259201sec
 Y : 26.68615160112359 , Cb : 9.214939570564528 , Cr : 11.244378922988902 
TIME TAKEN FOR ALPHA = 0.4 is 504.54076766967773sec
 Y : 28.33703507025613 , Cb : 6.637202890492865 , Cr : 14.910007142762518 
TIME TAKEN FOR ALPHA = 0.5 is 503.9280219078064sec
 Y : 27.81879958811558 , Cb : 8.46239465991204 , Cr : 12.504613715976623 
TIME TAKEN FOR ALPHA = 0.6 is 504.1308455467224sec
 Y : 25.698666634840915 , Cb : 10.421866654312112 , Cr : 19.196511490695578 
TIME TAKEN FOR ALPHA = 0.7 is 518.877527475357sec
 Y : 23.41682183246797 , Cb : 9.282177892797364 , Cr : 15.531180467583269 
TIME TAKEN FOR ALPHA = 0.8 is 497.4582748413086sec
 Y : 21.363958339040952 , Cb : 10.618252986270315 , Cr : 20.593

In [25]:
for alpha in [1.1,1.2,1.3,1.4,1.5]:
    start = time.time()
    
    hist_Y = Completed_LBP(images1,images2,0,sizes,alpha)
    hist_Cb = Completed_LBP(images1,images2,1,sizes,alpha)
    hist_Cr = Completed_LBP(images1,images2,2,sizes,alpha)

    same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
    same_pens_Y = [ v for v in same_pens_Y. values() ]
    diff_pens_Y = [ v for v in diff_pens_Y. values() ]

    same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
    same_pens_Cb = [ v for v in same_pens_Cb. values() ]
    diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

    same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
    same_pens_Cr = [ v for v in same_pens_Cr. values() ]
    diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

    dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
    dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
    dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)
    
    print('TIME TAKEN FOR ALPHA = {} is {}sec'.format(alpha,time.time()-start))
    print(' Y : {} , Cb : {} , Cr : {} '.format(dy,dcb,dcr))
    distances_Y.append(dy)
    distances_Cb.append(dcb)
    distances_Cr.append(dcr)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


TIME TAKEN FOR ALPHA = 1.1 is 511.8604373931885sec
 Y : 14.832179036813079 , Cb : 11.543277925909146 , Cr : 15.8369733333399 
TIME TAKEN FOR ALPHA = 1.2 is 3838.4412615299225sec
 Y : 12.86630679888612 , Cb : 8.101663618973474 , Cr : 14.903162306392643 
TIME TAKEN FOR ALPHA = 1.3 is 513.9898271560669sec
 Y : 11.082352179628977 , Cb : 8.993563119346652 , Cr : 11.671908060655168 
TIME TAKEN FOR ALPHA = 1.4 is 508.19417452812195sec
 Y : 8.881376274398466 , Cb : 6.94090667676012 , Cr : 9.364341124630068 
TIME TAKEN FOR ALPHA = 1.5 is 542.7149620056152sec
 Y : 7.268885439766813 , Cb : 5.742754329667475 , Cr : 10.915347235398306 
